In [2]:
import pandas as pd
import os
import numpy as np

In [7]:
station = 23025122
features = ['tempsolocm40cm']

In [4]:
# 2020-08-16
def date_to_season(date):
	'''
	Começa na primavera, termina no verão
 	'''
	ano, mes, dia = date.split('-')
	mes, dia = int(mes), int(dia)
	if (mes > 3 and mes < 6) or (mes == 3 and dia >= 21) or (mes == 6 and dia < 21):
		# outono
		return 1
	if (mes > 6 and mes < 9) or (mes == 6 and dia >= 21) or (mes == 9 and dia < 23):
		# inverno
		return 2
	if (mes > 9 and mes < 12) or (mes == 9 and dia >= 23) or (mes == 12 and dia < 21):
		# primavera
		return 3
	else:
		# verao
		return 0

In [8]:
data = pd.read_csv('../Dados/por estacao/{}/export_automaticas_{}_{}.csv'.format(station, station, features[0]))
for feature in features[1:]:
	data[feature] = pd.read_csv('../Dados/por estacao/{}/export_automaticas_{}_{}.csv'.format(station, station, feature))[feature]
for estacao_ano in range(4):
	data_estacao = data.loc[lambda data:data['date'].apply(lambda dia: date_to_season(dia) == estacao_ano)]
	date_dict = {}
	for i, date in enumerate(data_estacao.groupby(['date'])):
		date_dict[date[0]] = i
	day_station_id = [date_dict[date] for date in data_estacao['date'].to_list()]
	global_time = [int(int(h.split(':')[1])/15) + int(h.split(':')[0]) * 4 for h in data_estacao['hour'].to_list()]

	feature_data = {}
	for feature in features:
		data_list = data_estacao[feature].to_list()
		feature_data[feature] = [d if d and not np.isnan(d) else data_list[i-1] for i, d in enumerate(data_list)]

	month = [dia.split('-')[1] for dia in data_estacao['date'].to_list()]
	hour = [int(h.split(':')[0]) for h in data_estacao['hour'].to_list()]
	year = [int(d.split('-')[0]) for d in data_estacao['date'].to_list()]
	
	new_data = pd.DataFrame({
		'Day_Station_ID': day_station_id,
		'Global_Time': global_time,
		'month': month,
		'hour': hour,
		'year': year,
		'season': estacao_ano
	})
	for feature in features:
		new_data[feature] = feature_data[feature]
			
	arq_name = '.'.join(features)
	try:
		parent_dir = "./data/seasons/"			 
		os.makedirs(os.path.join(parent_dir, arq_name))
	except:
		pass

	new_data.to_csv('./data/seasons/{}/{}.{}.dia_id.season_{}.csv'.format(arq_name, arq_name, station, estacao_ano), index=False)